In [4]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import holodeck as holo
import holodeck.single_sources as ss
import matplotlib.pyplot as plt
import scipy as sp

In [3]:
edges, number, fobs, exname = ss.example4(print_test=False)
hc_bg, hc_ss, ssidx, hsamp, bgpar, sspar = ss.ss_by_cdefs(edges, number, 30, params=True)

11:46:45 INFO : zero_coalesced=True, zero_stalled=False [sam.py:dynamic_binary_number]
11:46:45 INFO : Stalled SAM bins based on GMT: 6.25e+03/1.56e+04 = 4.00e-01 [sam.py:static_binary_density]
11:46:45 INFO : Adding MMbulge scatter (3.4000e-01) [sam.py:static_binary_density]
11:46:45 INFO : 	dens bef: (0.00e+00, 5.88e-204, 7.46e-47, 1.42e-04, 6.93e-03, 1.16e-02, 1.54e-02) [sam.py:static_binary_density]
11:46:45 INFO : Scatter added after 0.161583 sec [sam.py:static_binary_density]
11:46:45 INFO : 	dens aft: (3.56e-18, 6.31e-13, 2.57e-10, 3.25e-04, 6.99e-03, 1.16e-02, 1.41e-02) [sam.py:static_binary_density]
11:46:45 INFO : 	mass: 2.38e-02 ==> 2.37e-02 || change = -6.2202e-03 [sam.py:static_binary_density]
11:46:45 INFO : zeroing out 6.25e+03/1.56e+04 = 4.00e-01 systems stalled from GMT [sam.py:static_binary_density]
11:46:45 INFO : fraction of coalesced binaries: 4.04e+04/1.88e+05 = 2.16e-01 [sam.py:dynamic_binary_number]
11:46:45 WARNING : WARNING: _stalled_ binaries are not being ac

# Rosado et al 2015

## For GWB
PDF in the absence of a GWB
$$ p_0(S) = \frac{1}{\sqrt{2\pi\sigma_0^2} }e^{-\frac{(S-\mu_0)^2}{2\sigma_0^2}} $$
PDF if GWB is present in the data
 $$ p_1(S) = \frac{1}{\sqrt{2\pi\sigma_1^2} }e^{-\frac{(S-\mu_1)^2}{2\sigma_1^2}} $$

cross correlation
$$ S = \int_{-T/2}^{T/2} dt \int_{-T/2}^{T/2} dt' s_i(t) s_j(t') Q(t,t')$$
$T$ = observing time, $s_i(t)$ and $s_j(t)$ are different pulsar data, $Q(t,t')$ = filter function, chosen to maximize DP for a fixed FAP $=\alpha_0=0.001$ (Neyman-Pearson criterion)


## For single sources
PDF of $\mathcal{F}_e$ - statistic in absence of signal
$$ p_0(\mathcal{F}_e) = \mathcal{F}_e e^{-\mathcal{F}_e}$$
PDF of $\mathcal{F}_e$ - statistic if signal is present
$$ p_1(\mathcal{F}_e, \rho) = \frac{(2\mathcal{F}_e)^{1/2}}{\rho} I_1 (\rho \sqrt{2 \mathcal{F}_e}) e^{-\mathcal{F}_e - 1/2\rho^2} $$
where $I_1(x)$ = the modified Bessel function of the first kind of order 1 and
 $\rho$ = the optimal $S/N_S = \big[ \sum_{i=1}^M S/N_i^2 \big]^{1/2}$

## Detection probability
for single sources: $ \gamma_S(t) = \int_0^t p_s(t') dt'$ \
for background: $ \gamma_B(t) = \int_0^t p_B(t') dt'$

# Variables
* $\alpha$ = false alarm probability
* $\gamma$ = detection probability
* $S_T$ = threshold signal

$$ \gamma_{bg} = \frac{1}{2} \mathrm{erfc} \big[ \frac{\sqrt{2} \sigma_0 \mathrm{erfc}^{-1}(2\alpha_0) - \mu_1}{\sqrt{2} \sigma_1}\big]

In [ ]:
ALPHA0 = 0.001 # false alarm probability (FAP)
GAMMA0 = 0.95 # detection probability

def bg_detection_probability(sigma_0, sigma_1, mu_1, alpha_0):
    """ 
    background detetion probability gamma_bg
    """
    temp = ((np.sqrt(2) * sigma_0 * sp.erfinv(2*alpha_0) - mu_1)
            /(np.sqrt(2) * sigma_1))
    gamma_bg = .5 * sp.erf(temp)
    return gamma_bg

def S_NA(mu_1, sigma_0):
    """ 
    SNR for the A-statistic S/N_A
    """
    return mu_1/sigma_0

def S_NB(mu_1, sigma_1):
    """ 
    SNR for the B-statistic S/N_B
    """
    return mu_1/sigma_1


threshold signal to noise to have FAP $\alpha <\alpha_0$ and detection probability $\gamma > \gamma_0$

$$ \mathrm{S/N^T} \approx \sqrt{2} \big[ \mathrm{erfc}^{-1}(2\alpha_0) - \mathrm{erfc}^{-1}(2 \gamma_0) \big] $$

In [7]:
def threshold_SNR(alpha_0, gamma_0):
    """ 
    threshold signal to noise ratio S/N^T for alpha<alpha_0 and gamma > gamma_0
    """
    return np.sqrt(2) * (sp.erfinv(2*alpha_0) - sp.erfinv(2*gamma_0))

$$ \Gamma_{ij} = \frac{3}{2} \gamma_{ij} \ln (\gamma_{ij}) - \frac{1}{4} \gamma_{ij} + \frac{1}{2} + \frac{1}{2}\delta_{ij} $$
$$ \gamma_{ij} = [1-\cos (\theta_{ij})]/2$$

In [8]:
def gammaij_from_thetaij(theta_ij):
    """ 
    gamma_ij for two pulsars of relative angle theta_ij
    """
    return (1-np.cos(theta_ij))/2

def thetaij_from_thetai_thetaj(theta_i, theta_j):
    """ 
    relative angle between two pulsars with angular positions theta_i and theta_j
    """
    return np.abs(theta_i - theta_j)

def dirac_ij(i,j):
    """ 
    the dirac delta function of i,j
    """
    if(i==j): return 1
    else: return 0

def Gammaij_from_gammaij_diracij(gamma_ij, dirac_ij):
    """ 
    Gamma_i,j as a function of gamma_i,j and diracdelta_i,j
    """
    return (3/2 * gamma_ij *np.log(gamma_ij)
            - 1/4 * gamma_ij
            + 1/2 + dirac_ij)

def Gammaij_from_thetai_thetaj_ij(theta_i, theta_j, i, j):
    """ 
    Gamma_i,j as a function of theta_i, theta_j, i, and j
    """
    dirac_ij = dirac_ij(i, j)
    theta_ij = thetaij_from_thetai_thetaj(theta_i, theta_j)
    gamma_ij = gammaij_from_thetaij(theta_ij)
    return Gammaij_from_gammaij_diracij(gamma_ij, dirac_ij)